# Week 3 — Robot Dynamics (Expanded)

This notebook derives the dynamics of a planar 2-DOF manipulator and demonstrates PD and computed-torque controllers. It uses `robot_utils.py` and `plot_utils.py` included in this folder.

## 1) Symbolic derivation (SymPy)

In [ ]:
# Symbolic derivation of M(q), C(q,dq), G(q)
import sympy as sp
from robot_utils import make_symbolic_2link
s = make_symbolic_2link()
q1 = s['q1']; q2 = s['q2']; dq1 = s['dq1']; dq2 = s['dq2']
l1 = s['l1']; l2 = s['l2']; m1 = s['m1']; m2 = s['m2']; I1 = s['I1']; I2 = s['I2']; g = s['g']
T = s['T']; V = s['V']
# Build inertia matrix
qs = [q1,q2]; dqs = [dq1,dq2]
M = sp.zeros(2,2)
for i in range(2):
    for j in range(2):
        M[i,j] = sp.simplify(sp.diff(sp.diff(T, dqs[i]), dqs[j]))
# Christoffel and C
Gamma = [[[None for _ in range(2)] for __ in range(2)] for ___ in range(2)]
for k in range(2):
    for i in range(2):
        for j in range(2):
            Gamma[k][i][j] = sp.Rational(1,2)*(sp.diff(M[k,j], qs[i]) + sp.diff(M[k,i], qs[j]) - sp.diff(M[i,j], qs[k]))
C = sp.zeros(2,2)
for i in range(2):
    for j in range(2):
        ssum = 0
        for k in range(2):
            ssum += Gamma[i][j][k] * dqs[k]
        C[i,j] = sp.simplify(ssum)
G = sp.Matrix([sp.diff(V, qi) for qi in qs])

M_s = sp.simplify(M)
C_s = sp.simplify(C)
G_s = sp.simplify(G)

print('Derived symbolic M, C, G')


## 2) Lambdify and numeric evaluation

In [ ]:
# Lambdify
from robot_utils import lambdify_dynamics
M_f, C_f, G_f = lambdify_dynamics(M_s, C_s, G_s)

# numeric test
import numpy as np
params = dict(l1=1.0, l2=0.8, m1=1.0, m2=0.8, I1=0.01, I2=0.01, g=9.81)
qtest = (0.3, 0.4); dqtest = (0.1, -0.2)
Mnum = np.array(M_f(qtest[0], qtest[1], params['l1'], params['l2'], params['m1'], params['m2'], params['I1'], params['I2']), dtype=float)
Cnum = np.array(C_f(qtest[0], qtest[1], dqtest[0], dqtest[1], params['l1'], params['l2'], params['m1'], params['m2'], params['I1'], params['I2']), dtype=float)
Gnum = np.array(G_f(qtest[0], qtest[1], params['l1'], params['l2'], params['m1'], params['m2'], params['g']), dtype=float).reshape(2,)
print('M num =', Mnum)
print('C num =', Cnum)
print('G num =', Gnum)


## 3) PD baseline and computed-torque controller (simulation)

In [ ]:
# Simulation comparing PD and computed-torque controllers
import numpy as np
from scipy.integrate import solve_ivp
from plot_utils import plot_joint_trajectories, plot_end_effector_path

p = dict(l1=1.0, l2=0.8, m1=1.0, m2=0.8, I1=0.01, I2=0.01, g=9.81)
M_func = lambda qv: np.array(M_f(qv[0], qv[1], p['l1'], p['l2'], p['m1'], p['m2'], p['I1'], p['I2']), dtype=float)
C_func = lambda qv,dqv: np.array(C_f(qv[0], qv[1], dqv[0], dqv[1], p['l1'], p['l2'], p['m1'], p['m2'], p['I1'], p['I2']), dtype=float)
G_func = lambda qv: np.array(G_f(qv[0], qv[1], p['l1'], p['l2'], p['m1'], p['m2'], p['g']), dtype=float).reshape(2,)

# controllers
Kp_pd = np.diag([80.,60.]); Kd_pd = np.diag([8.,6.])
Kp_ct = np.diag([100.,80.]); Kd_ct = np.diag([20.,15.])
q_des = np.array([np.pi/4, np.pi/6]); dq_des = np.array([0.,0.]); ddq_des = np.array([0.,0.])

def dyn_pd(t,y):
    qv = y[:2]; dqv = y[2:]
    M_eval = M_func(qv); C_eval = C_func(qv,dqv); G_eval = G_func(qv)
    tau = Kp_pd.dot(q_des - qv) + Kd_pd.dot(dq_des - dqv)
    ddq = np.linalg.solve(M_eval, tau - C_eval.dot(dqv) - G_eval)
    return np.concatenate((dqv, ddq))

def dyn_ct(t,y):
    qv = y[:2]; dqv = y[2:]
    M_eval = M_func(qv); C_eval = C_func(qv,dqv); G_eval = G_func(qv)
    v = ddq_des + Kd_ct.dot(dq_des - dqv) + Kp_ct.dot(q_des - qv)
    tau = M_eval.dot(v) + C_eval.dot(dqv) + G_eval
    ddq = np.linalg.solve(M_eval, tau - C_eval.dot(dqv) - G_eval)
    return np.concatenate((dqv, ddq))

y0 = np.zeros(4)
T = np.linspace(0,5,400)
sol_pd = solve_ivp(dyn_pd, (0,5), y0, t_eval=T)
sol_ct = solve_ivp(dyn_ct, (0,5), y0, t_eval=T)

# plots
plot_joint_trajectories(T, sol_pd.y, labels=('q1 PD','q2 PD'))
plot_joint_trajectories(T, sol_ct.y, labels=('q1 CT','q2 CT'))

from matplotlib import pyplot as plt
plt.show()

# end-effector paths
plot_end_effector_path(sol_pd.y[:2,:], p['l1'], p['l2'], title='PD End-effector')
plot_end_effector_path(sol_ct.y[:2,:], p['l1'], p['l2'], title='Computed-Torque End-effector')
plt.show()
